In [14]:

import warnings
warnings.filterwarnings('ignore')

In [15]:
import math

import numpy as np
import pandas as pd

from backtesting import Backtest
from backtesting.lib import TrailingStrategy
from backtesting.test import SMA
import tulipy as tu


In [16]:
dataset_path = '/home/paolo/PycharmProjects/ng_strategies/data/HistoricalData_NG.csv'

In [17]:
class MomentumStrategy(TrailingStrategy):

    n = 4
    buy_open = False
    sell_open = False
    order_price = None

    def init(self):
        super().init()
        self.mom = self.I(tu.mom, self.data.Close, 1)
        self.set_trailing_sl(1.5)

    @staticmethod
    def get_volume_signal(volumes, n):
        return np.mean(volumes[-1:-n]) > np.mean(volumes[-1:-10])

    def next(self):

        skip = False

        try:
            self.data.Close[-self.n]
        except IndexError:
            super(MomentumStrategy, self).next()
            skip = True
        except Exception:
            skip = True

        if not skip:
            if self.buy_open:
                if self.data.High >= self.order_price*1.10:
                    self.position.close()

            if self.sell_open:
                if self.data.Low <= self.order_price*0.90:
                    self.position.close()

            if self.data.High[-1] > self.data.Close[-self.n] and \
                    self.get_volume_signal(self.data.Volume, self.n):
                if self.sell_open:
                    self.position.close()
                    self.sell_open = False

                self.buy(size=math.floor(self.equity*0.1/self._broker.last_price))
                self.buy_open = True

            if self.data.Low[-1] < self.data.Close[-self.n] and \
                    self.get_volume_signal(self.data.Volume, self.n):
                if self.buy_open:
                    self.position.close()
                    self.buy_open = False

                self.sell(size=math.floor(self.equity*0.1/self._broker.last_price))
                self.sell_open = True

In [18]:
data = pd.read_csv(dataset_path, sep=',', index_col=0, parse_dates=True)
data = data.dropna()

data = data.set_index(pd.to_datetime(data.index))
data.sort_values(by='Date', inplace=True)

In [22]:
bt = Backtest(data, MomentumStrategy, cash=100_000, commission=0.02, trade_on_close=True)
stats = bt.run()

stats

Traceback (most recent call last):
  File "/snap/pycharm-professional/254/plugins/python/helpers-pro/jupyter_debug/pydev_jupyter_utils.py", line 64, in attach_to_debugger
    debugger.prepare_to_run(enable_tracing_from_start=False)
TypeError: prepare_to_run() got an unexpected keyword argument 'enable_tracing_from_start'
Failed to connect to target debugger.


Start                     2011-08-31 00:00:00
End                       2021-08-30 00:00:00
Duration                   3652 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                     100000.0
Equity Peak [$]                      100000.0
Return [%]                                0.0
Buy & Hold Return [%]                6.191416
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

In [20]:
bt.plot()

Row(id='1863', ...)